In [ ]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from matplotlib.ticker import MaxNLocator

# ========== Path settings ==========
BASE_DIR = r"C:\Users\**\**\**\PFAS"
INPUT_FILENAME = "input.xlsx"
INPUT_PATH = os.path.join(BASE_DIR, INPUT_FILENAME)
OUT_DIR = BASE_DIR
os.makedirs(OUT_DIR, exist_ok=True)

# ========== Load input file ==========
def load_input(path: str) -> pd.DataFrame:
    ext = os.path.splitext(path)[1].lower()
    if ext in [".xlsx", ".xls"]:
        return pd.read_excel(path)
    elif ext == ".csv":
        return pd.read_csv(path)
    else:
        raise ValueError("Only .xlsx / .xls / .csv input files are supported.")

df_raw = load_input(INPUT_PATH)

# ========== Required column names (English only) ==========
NAME_COL = "name"

PI3K_COL = "P_PI3K_AKT"
PPAR_COL = "P_PPAR"
LPS_COL  = "P_LPS"
ROS_COL  = "P_ROS"
POA_COL  = "P_OA"

AVAILABLE_COLS = df_raw.columns.tolist()

if NAME_COL not in AVAILABLE_COLS:
    raise ValueError(f"Required column '{NAME_COL}' not found.")

PATH_MAP = []
if PI3K_COL in AVAILABLE_COLS:
    PATH_MAP.append(("PI3K", "PI3K–AKT", PI3K_COL))
if PPAR_COL in AVAILABLE_COLS:
    PATH_MAP.append(("PPAR", "PPAR signaling", PPAR_COL))
if LPS_COL in AVAILABLE_COLS:
    PATH_MAP.append(("LPS", "LPS", LPS_COL))
if ROS_COL in AVAILABLE_COLS:
    PATH_MAP.append(("ROS", "ROS", ROS_COL))
if POA_COL in AVAILABLE_COLS:
    PATH_MAP.append(("POA", "P(OA)", POA_COL))

if len(PATH_MAP) == 0:
    raise ValueError("No valid pathway probability columns found.")

# ========== Data cleaning ==========
df = df_raw.copy()
df.rename(columns={NAME_COL: "name"}, inplace=True)

for key, _, src_col in PATH_MAP:
    df[key] = (
        pd.to_numeric(df[src_col], errors="coerce")
        .fillna(0.0)
        .clip(0.0, 1.0)
    )

df = df.dropna(subset=["name"]).reset_index(drop=True)

# ========== Utilities ==========
def minmax01(arr):
    arr = np.asarray(arr, dtype=float)
    lo, hi = np.min(arr), np.max(arr)
    if hi - lo < 1e-12:
        return np.zeros_like(arr)
    return (arr - lo) / (hi - lo)

def fit_gmm_scan(values, k_range=range(2, 7), random_state=42):
    x = values.reshape(-1, 1)

    if np.allclose(x, x[0]):
        gm = GaussianMixture(n_components=1, random_state=random_state).fit(x)
        table = pd.DataFrame([[1, np.nan, np.nan]], columns=["K", "BIC", "AIC"])
        return gm, 1, np.array([0]), table, {"choice": "constant input"}

    rows, models = [], {}
    best_bic, best_aic = np.inf, np.inf
    K_bic, K_aic = None, None

    for k in k_range:
        gm = GaussianMixture(
            n_components=k,
            covariance_type="full",
            random_state=random_state,
            n_init=10
        ).fit(x)
        bic, aic = gm.bic(x), gm.aic(x)
        rows.append([k, bic, aic])
        models[k] = gm
        if bic < best_bic:
            best_bic, K_bic = bic, k
        if aic < best_aic:
            best_aic, K_aic = aic, k

    table = pd.DataFrame(rows, columns=["K", "BIC", "AIC"])
    table["score"] = 0.6 * minmax01(table["BIC"]) + 0.4 * minmax01(table["AIC"])

    if K_bic == K_aic:
        chosen = K_bic
        reason = "BIC and AIC agree"
    else:
        chosen = table.loc[table["score"].idxmin(), "K"]
        reason = "minimum combined score"

    gm_final = models[chosen]
    comp_order = np.argsort(gm_final.means_.flatten())

    return gm_final, chosen, comp_order, table, {"choice": reason}

def assign_levels_and_probs(x, gm, comp_order):
    x = x.reshape(-1, 1)
    resp = gm.predict_proba(x)

    probs_sorted = np.zeros_like(resp)
    for lvl, comp_idx in enumerate(comp_order):
        probs_sorted[:, lvl] = resp[:, comp_idx]

    levels = 1 + np.argmax(probs_sorted, axis=1)
    return levels, probs_sorted

# ========== Plotting ==========
def plot_gmm_density(title, x, gm, comp_order, save_path):
    xs = np.linspace(x.min(), x.max(), 400).reshape(-1, 1)

    weights = gm.weights_[comp_order]
    means = gm.means_.flatten()[comp_order]
    vars_ = gm.covariances_.flatten()[comp_order]

    def pdf(xx, mu, var):
        return 1.0 / np.sqrt(2 * np.pi * var) * np.exp(-(xx - mu) ** 2 / (2 * var))

    plt.figure(figsize=(7, 4))
    plt.hist(x, bins=30, density=True, alpha=0.35)

    for i in range(len(weights)):
        plt.plot(xs, weights[i] * pdf(xs, means[i], vars_[i]),
                 label=f"Level {i+1}")

    plt.xlabel(f"{title} probability")
    plt.ylabel("Density")
    plt.title(f"GMM density – {title}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

# ========== Main workflow ==========
summary = []

for key, title, _ in PATH_MAP:
    x = df[key].values
    gm, K, order, ic_table, info = fit_gmm_scan(x)

    levels, probs = assign_levels_and_probs(x, gm, order)

    out = pd.DataFrame({
        "name": df["name"],
        title: x,
        f"Level_{key}": levels
    })

    for i in range(probs.shape[1]):
        out[f"Prob_Level{i+1}"] = probs[:, i]

    out.to_csv(os.path.join(OUT_DIR, f"GMM_list_{key}.csv"),
               index=False, encoding="utf-8-sig")

    plot_gmm_density(
        title, x, gm, order,
        os.path.join(OUT_DIR, f"GMM_density_{key}.png")
    )

    vals, cnts = np.unique(levels, return_counts=True)
    for v, c in zip(vals, cnts):
        summary.append([title, v, c, K, info["choice"]])

summary_df = pd.DataFrame(
    summary,
    columns=["Pathway", "Level", "Count", "K_best", "Selection_reason"]
)
summary_df.to_excel(
    os.path.join(OUT_DIR, "GMM_summary.xlsx"),
    index=False
)

print("GMM stratification completed. All outputs saved to:", OUT_DIR)

